In [39]:
from theano import *
import theano.tensor as T

In [38]:
x = T.dscalar('x')

In [13]:
y = T.dscalar('y')

In [14]:
z = x + y

In [15]:
f = function([x, y], z)

In [16]:
f(2, 3)

array(5.0)

In [18]:
f(16.3, 12.1)

array(28.4)

In [19]:
x = T.dmatrix('x')

In [20]:
y = T.dmatrix('y')

In [21]:
z = x + y

In [22]:
f = function([x, y], z)

In [23]:
f([[1, 2], [3, 4]], [[10, 20], [30, 40]])

array([[ 11.,  22.],
       [ 33.,  44.]])

In [24]:
a = T.vector()
out = a + a ** 10
f = function([a], out)
print(f([0, 1, 2]))

[    0.     2.  1026.]


In [29]:
a = T.dvector()
b = T.dvector()
out = a ** 2 + b ** 2 + 2 * a * b
f = fn([a, b], out)
print(f([1, 2], [4, 5]))

[ 25.  49.]


In [42]:
x = T.dmatrix('x')

In [43]:
s = 1 / (1 + T.exp(-x))

In [44]:
logistic = function([x], s)

In [45]:
logistic([[0, 1], [-1, -2]])

array([[ 0.5       ,  0.73105858],
       [ 0.26894142,  0.11920292]])

In [46]:
s2 = (1 + T.tanh(x / 2)) / 2

In [47]:
logistic2 = function([x], s2)

In [48]:
logistic2([[0, 1], [-1, -2]])

array([[ 0.5       ,  0.73105858],
       [ 0.26894142,  0.11920292]])

In [49]:
a, b = T.dmatrices('a', 'b')
diff = a - b
abs_diff = abs(diff)
diff_squared = diff**2
f = function([a, b], [diff, abs_diff, diff_squared])
f([[1, 1], [1, 1]], [[0, 1], [2, 3]])

[array([[ 1.,  0.],
        [-1., -2.]]), array([[ 1.,  0.],
        [ 1.,  2.]]), array([[ 1.,  0.],
        [ 1.,  4.]])]

In [50]:
x, y = T.dscalars('x', 'y')
z = x + y
f = function([x, Param(y, default=1)], z)

In [51]:
f(33)

array(34.0)

In [52]:
f(33, 2)

array(35.0)

In [53]:
state = shared(0)

In [54]:
inc = T.iscalar('inc')

In [55]:
accumulator = function([inc], state, updates=[(state, state+inc)])

In [56]:
state.get_value()

array(0)

In [57]:
accumulator(1)

array(0)

In [58]:
state.get_value()

array(1)

In [59]:
accumulator(300)

array(1)